## Dependencies

In [1]:
# Confirmed needed dependencies
import zillow
import random
import pprint

# Seth 

# Research how to make this truly invisible on GitHub Repo
# from config import zwsid
# Paste in your zwsid in the meantime using the following line
# zwsid = ""

In [2]:
# Possibly needed, but not yet used dependencies
import pandas as pd
import requests
import json

In [ ]:
# Valerie

# Further setting up Notebook with Markdown sections, file outputs, action comments, etc.
# Create pandas dataframe 

## Geocoordinates of Austin

In [ ]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

# dependencies for geocoordinates generator
import random
import sys
import math

In [ ]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 10         #### Set radius in miles

points = 1000        #### Set number of lat,lon points to generate

output_file_name = "random_Lat_Lon.csv" #### Set output file name 

In [ ]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points = points+1


In [ ]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

In [ ]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(output_file_name, index=False, header=True)

## Plot coordinate points on map

Troy is working on this as a way to verify good random sampling of coordinate points and area selection, and as a way to get familar with plotting on maps


In [ ]:

import gmplot

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

#gmap.plot(latitudes, longitudes, 'cornflowerblue', edge_width=10)
gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"], '#3B0B39', size=10, marker=False)
#gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"] , 'k', marker=True)
#gmap.heatmap(, heat_lngs)

gmap.draw("mymap.html")

## Convert Coordinates to Residential Addresses

In [3]:
##### Yuta Blocks #####

# Testing Geopy Nominatim API Response
import geopy
from geopy.geocoders import Nominatim

geopy.geocoders.options.default_user_agent = "my-application"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

# Params 1 - Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== #01 - Home Response:")
response_1 = requests.get(url, params=params_1).json()
pp.pprint(response_1)
print("\n" + "="*60 + "\n")

===== #01 - Home Response:
{   'address': {   'country': 'USA',
                   'country_code': 'us',
                   'county': 'Williamson County',
                   'hamlet': 'Pond Springs',
                   'house_number': '12126',
                   'neighbourhood': "Hunter's Chase",
                   'postcode': '78729',
                   'road': 'Hunters Chase Drive',
                   'state': 'Texas',
                   'suburb': 'Jollyville'},
    'addresstype': 'place',
    'boundingbox': ['30.440962', '30.441162', '-97.777478', '-97.777278'],
    'category': 'place',
    'display_name': "12126, Hunters Chase Drive, Hunter's Chase, Jollyville, "
                    'Pond Springs, Williamson County, Texas, 78729, USA',
    'importance': '0',
    'lat': '30.441062',
    'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
               'https://osm.org/copyright',
    'lon': '-97.777378',
    'name': None,
    'osm_id': '3825454876',
    'osm_type': 'node',
 

In [4]:
# Import CSV, put into DataFrame
data_file = "random_Lat_Lon_sample.csv"
latlon_df = pd.read_csv(data_file)
latlon_df.head()

,lat,lon
0,30.291394,-97.852524
1,30.194187,-97.625975
2,30.240834,-97.777685
3,30.328267,-97.641748
4,30.239495,-97.862239


In [5]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

In [6]:
# Loop Request API and append to create new dataframe
query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
aType = []
lat = []
lon = []

counter = 1
numRequests = len(lats)

for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }
    
    response = requests.get(query_url, params=params).json()
    
    if response['type'] == 'house' or response['type'] == 'yes':
        postcode.append(response['address']['postcode'])
        aType.append(response['type'])
        lat.append(response['lat'])
        lon.append(response['lon'])
    
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
    
        print(f"Processed Record {counter} of {numRequests}.")
        counter = counter + 1
    else:
        print(f"Skipped Record {counter} of {numRequests} (wrong type or missing field).")
        counter = counter + 1

Processed Record 1 of 99.
Skipped Record 2 of 99 (wrong type or missing field).
Skipped Record 3 of 99 (wrong type or missing field).
Processed Record 4 of 99.


SSLError: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?format=jsonv2&lat=30.23949535&lon=-97.86223905 (Caused by SSLError(SSLError("bad handshake: SysCallError(60, 'ETIMEDOUT')",),))

In [ ]:
# Create a dataframe with all addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
    "type": aType,
    "lat": lat,
    "lon": lon
})
address_df.head()

In [ ]:
# Kat's block

## Zillow API Calls using Address and Zipcode

In [ ]:
address = "929 Cavalier Ln" 
zipcode = "78757"

pp = pprint.PrettyPrinter(indent=4)

api = zillow.ValuationApi()


#This appears to be the ideal search to use with the most flushed out info
deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
pp.pprint(deep_results.get_dict())


# This appears to return the same as .GetZEstimate(),
# but has less accurate "extended_data" than .GetDeepSearchResults
# data = api.GetSearchResults(zwsid, address, zipcode)
# pp.pprint(data.get_dict())


# This appears to return the same as .GetSearchResults()
# detail_data = api.GetZEstimate(zwsid, data.zpid)
# pp.pprint(detail_data.get_dict())

## Calculate Value per Sqft

In [ ]:
# Seth

#Create function to calculate value per sqft using "zestimate"["amount"] / "extended_data"["finished_sqft"]
